In [2]:
"""
데이터를 MYSQL에서 불러온다.(E)

데이터 가공 (T)

데이터를 Postgres에 저장 (L)
"""

'\n데이터를 MYSQL에서 불러온다.(E)\n\n데이터 가공 (T)\n\n데이터를 Postgres에 저장 (L)\n'

## 1. 절차 지향적 데이터 파이프라인 설계

In [3]:
import dotenv
import os

env_path = dotenv.find_dotenv()
dotenv.load_dotenv(env_path)

DB_SETTINGS = dict(
    mysql_params = dict(
        engine_name = os.getenv('MYSQL_ENGINE_NAME', ""),
        user = os.getenv('MYSQL_USER', ""),
        password = os.getenv('MYSQL_PASSWORD', ""),
        host = os.getenv('MYSQL_HOST', ""),
        port = os.getenv('MYSQL_PORT', ""),
        database = os.getenv('MYSQL_DATABASE', "")
    ),
    pg_params = dict(
        engine_name = os.getenv('PG_ENGINE_NAME', ""),
        user = os.getenv('PG_USER', ""),
        password = os.getenv('PG_PASSWORD', ""),
        host = os.getenv('PG_HOST', ""),
        port = os.getenv('PG_PORT', ""),
        database = os.getenv('PG_DATABASE', "")
    )
)

DB_SETTINGS

{'mysql_params': {'engine_name': 'mysql+pymysql',
  'user': 'root',
  'password': '123456',
  'host': 'localhost',
  'port': '3300',
  'database': 'docker_mysql'},
 'pg_params': {'engine_name': 'postgresql',
  'user': 'codeit',
  'password': 'sprint',
  'host': 'localhost',
  'port': '5430',
  'database': 'docker_postgres'}}

In [4]:
import pymysql, psycopg2
from sqlalchemy import create_engine

class DBConnector:
    def __init__(self, engine_name, user, password, host, port, database):
        self.engine_name = engine_name
        self.user = user
        self.password = password
        self.host = host
        self.port = port
        self.database = database
        
    # PYMYSQL 연결
    # DDL, DCL, DML -> MYSQL
    def pymysql_connection(self):
        pymysql_conn = pymysql.connect(
            user=self.user,
            password=self.password,
            host=self.host,
            port=int(self.port),
            database=self.database,
            charset='utf8'
            )
        return pymysql_conn
    
    # PSYCOPG2 연결
    # DDL, DCL, DML -> PG
    def psycopg2_connection(self):
        psycopg2_conn = psycopg2.connect(
            user=self.user,
            password=self.password,
            host=self.host,
            port=self.port,
            dbname=self.database,
            )
        return psycopg2_conn
    
    # SQLALCHEMY 연결
    # DDL, DCL, DML -> MYSQL, PG .....
    # 테이블을 PANDAS DF로 다루고 싶다! (read_sql, to_sql)
    def sqlalchemy_connection(self):
        sqlalchemy_conn = create_engine(f"{self.engine_name}://{self.user}:{self.password}@{self.host}:{self.port}/{self.database}")
        
        return sqlalchemy_conn
        

In [5]:
#데이터 추출 
import pandas as pd
# MYSQL 커넥션 불러오기

mysql_conn = DBConnector(**DB_SETTINGS['mysql_params']).sqlalchemy_connection()
query = "SELECT * FROM pokemon"
def extractor(connection_obj, query):
    df = pd.read_sql(
        sql = "SELECT * FROM pokemon",
        con = connection_obj
    )
    return df

extractor(mysql_conn,query)



id kor_name    eng_name    type1   type2  total   hp  attack  defense  \
0      1     이상해씨   Bulbasaur    Grass  Poison    318   45      49       49   
1      2     이상해풀     Ivysaur    Grass  Poison    405   60      62       63   
2      3     이상해꽃    Venusaur    Grass  Poison    525   80      82       83   
3      4      파이리  Charmander     Fire    None    309   39      52       43   
4      5      리자드  Charmeleon     Fire    None    405   58      64       58   
..   ...      ...         ...      ...     ...    ...  ...     ...      ...   
246  247     데기라스     Pupitar     Rock  Ground    410   70      84       70   
247  248     마기라스   Tyranitar     Rock    Dark    600  100     134      110   
248  249      루기아       Lugia  Psychic  Flying    680  106      90      130   
249  250      칠색조       Ho-oh     Fire  Flying    680  106     130       90   
250  251      세레비      Celebi  Psychic   Grass    600  100     100      100   

     special_attack  special_defense  speed  generation  is_legendary  
0                65               65     45           1             0  
1                80               80     60           1             0  
2               100              100     80           1             0  
3                60               50     65           1             0  
4                80               65     80           1             0  
..              ...              ...    ...         ...           ...  
246              65               70     51           2             0  
247              95              100     61           2             0  
248              90              154    110           2             1  
249             110              154     90           2             1  
250             100              100    100           2             0  

[251 rows x 14 columns]

In [6]:
# 데이터 가공

df = extractor(mysql_conn,query)

def transformer(pandas_df):
    t_df = df.value_counts('type1').to_frame().reset_index()
    return t_df

transformer(df)

type1  count
0      Water     46
1     Normal     37
2        Bug     22
3      Grass     21
4       Fire     20
5   Electric     15
6     Poison     15
7    Psychic     15
8       Rock     13
9     Ground     11
10  Fighting      9
11     Fairy      7
12       Ice      6
13      Dark      5
14     Ghost      4
15    Dragon      3
16     Steel      2

In [7]:
t_df = transformer(df)
pg_conn = DBConnector(**DB_SETTINGS['pg_params']).sqlalchemy_connection()

def loader( pandas_df, connection_obj, table_name):
    try:
        pandas_df.to_sql(
            name = 'pokemon_type',
            con =connection_obj,
            if_exists = 'replace',
            index = False
        )
        print("저장완료")
    except:
        print("저장실패")

loader(t_df, pg_conn, 'pokemon_type')
# 데이터 저장

저장완료


In [8]:
#컨트롤러 구현

def main():
    #파라미터 선언부
    mysql_conn = DBConnector(**DB_SETTINGS['mysql_params']).sqlalchemy_connection()
    pg_conn = DBConnector(**DB_SETTINGS['pg_params']).sqlalchemy_connection()  
    query = "SELECT * FROM pokemon"
    table_name = 'pokemon_type'

    # 함수 실행부
    df = extractor(mysql_conn,query)
    t_df = transformer(df)
    loader(t_df, pg_conn, 'pokemon_type')

main()

저장완료


In [9]:
#파이썬 모듈화

from settings import DB_SETTINGS
from db.connector import DBConnector
from pipeline.extract import extractor
from pipeline.transform import transformer
from pipeline.load import loader

In [10]:
def main():
    #파라미터 선언부
    mysql_conn = DBConnector(**DB_SETTINGS['mysql_params']).sqlalchemy_connection()
    pg_conn = DBConnector(**DB_SETTINGS['pg_params']).sqlalchemy_connection()  
    query = "SELECT * FROM pokemon"
    table_name = 'pokemon_type'

    # 함수 실행부
    df = extractor(mysql_conn,query)
    t_df = transformer(df)
    loader(t_df, pg_conn, 'pokemon_type')

main()

EXTRACTOR 시작!
TRANSFORMER 시작!
LOADER 시작!
POSTGRESQL에 테이블 저장 완료!
